# Word, Character, and BPE Tokenizers — A Minimal, Flow‑Based Walkthrough

Goal: **see each step** and keep code simple. We'll start with a tiny 5‑word corpus, then:
1) **Word‑level** tokenization (trivial split)
2) **Character‑level** tokenization (with `</w>` markers)
3) **BPE training** step‑by‑step (count pairs → merge top pair), one cell per step
4) **Use learned merges** to tokenize new words (greedy, minimal code)


In [1]:
from collections import Counter
from IPython.display import display
import pandas as pd

# Tiny 5-word corpus
corpus = ["low", "lower", "lowest", "newest", "widest"]
print("Corpus:", corpus)

# We'll also keep a small list of test words for later
test_words = ["lowest", "new", "widen", "lower", "slow"]
print("Test words:", test_words)

Corpus: ['low', 'lower', 'lowest', 'newest', 'widest']
Test words: ['lowest', 'new', 'widen', 'lower', 'slow']


## 1) Word‑level Tokenization (simple split)
For our tiny corpus, this is just the list of words. For a sentence, we can do `sentence.split()`.

In [2]:
sentence = "lowest new wid est"
word_tokens = sentence.split()
print("Sentence:", sentence)
print("Word tokens:", word_tokens)
print("# of tokens:", len(word_tokens))

Sentence: lowest new wid est
Word tokens: ['lowest', 'new', 'wid', 'est']
# of tokens: 4


## 2) Character‑level Tokenization (with `</w>` word end markers)
We convert each word to a list of characters and append `</w>` so BPE knows word boundaries.

In [7]:
# Build the initial symbol vocabulary as a dict: tuple(symbols) -> count
vocab = {}
for w in corpus:
    syms = tuple(list(w) + ["</w>"])
    vocab[syms] = vocab.get(syms, 0) + 1

print("Initial char-level vocab with </w> markers:")
for syms, cnt in vocab.items():
    print(" ", syms, "x", cnt)

print("\nAs a table (for readability):")
df_init = pd.DataFrame({
    "word_symbols": [" ".join(s) for s in vocab.keys()],
    "count": list(vocab.values())
}).sort_values(by=["count","word_symbols"], ascending=[False, True]).reset_index(drop=True)
display(df_init)

Initial char-level vocab with </w> markers:
  ('l', 'o', 'w', '</w>') x 1
  ('l', 'o', 'w', 'e', 'r', '</w>') x 1
  ('l', 'o', 'w', 'e', 's', 't', '</w>') x 1
  ('n', 'e', 'w', 'e', 's', 't', '</w>') x 1
  ('w', 'i', 'd', 'e', 's', 't', '</w>') x 1

As a table (for readability):


,word_symbols,count
0,l o w </w>,1
1,l o w e r </w>,1
2,l o w e s t </w>,1
3,n e w e s t </w>,1
4,w i d e s t </w>,1


## 3) BPE Training — **One merge per run** (repeat this cell)
Each time you run this cell:
1. Count frequencies of adjacent symbol pairs across the current `vocab`
2. Pick the **most frequent** pair
3. **Merge** it in every word (replace the pair with a single new symbol)
4. Append the merge to `merges` and print the updated state

> **Tip:** Run this cell multiple times (5–10) to watch merges evolve.

In [17]:
# Keep state across runs of this cell
try:
    merges
    step
except NameError:
    merges = []
    step = 0

step += 1
print(f"\n=== BPE Step {step} ===")

# 3.1 Count adjacent pairs
pairs = Counter()
for syms, freq in vocab.items():
    for a, b in zip(syms, syms[1:]):
        pairs[(a, b)] += freq

top = pairs.most_common(10)
print("Top pairs:")
for (a,b), f in top:
    print(f"  ({a} {b}) : {f}")

if not pairs:
    print("No more pairs to merge.")
else:
    # 3.2 Choose best pair
    (pa, pb), best_freq = pairs.most_common(1)[0]
    print(f"Merging most frequent pair: ({pa} {pb}) with freq = {best_freq}")

    # 3.3 Merge across vocab (replace (pa,pb) with pa+pb)
    merged_token = pa + pb
    new_vocab = {}
    for syms, freq in vocab.items():
        new_syms = []
        i = 0
        while i < len(syms):
            if i < len(syms)-1 and syms[i] == pa and syms[i+1] == pb:
                new_syms.append(merged_token)
                i += 2
            else:
                new_syms.append(syms[i])
                i += 1
        new_vocab[tuple(new_syms)] = new_vocab.get(tuple(new_syms), 0) + freq

    vocab = new_vocab
    merges.append((pa, pb))
    print("Updated merges:", merges)
    print("Updated vocab:")
    for syms, cnt in vocab.items():
        print(" ", syms, "x", cnt)

    # Show as a small table
    df_now = pd.DataFrame({
        "word_symbols": [" ".join(s) for s in vocab.keys()],
        "count": list(vocab.values())
    })
    display(df_now.sort_values(by=["count","word_symbols"], ascending=[False,True]).reset_index(drop=True))


=== BPE Step 12 ===
Top pairs:
  (low est</w>) : 1
  (n e) : 1
  (e w) : 1
  (w est</w>) : 1
  (w i) : 1
  (i d) : 1
  (d est</w>) : 1
Merging most frequent pair: (low est</w>) with freq = 1
Updated merges: [('l', 'o'), ('lo', 'w'), ('l', 'o'), ('lo', 'w'), ('e', 's'), ('es', 't'), ('est', '</w>'), ('low', '</w>'), ('low', 'e'), ('lowe', 'r'), ('lower', '</w>'), ('low', 'est</w>')]
Updated vocab:
  ('low</w>',) x 1
  ('lower</w>',) x 1
  ('lowest</w>',) x 1
  ('n', 'e', 'w', 'est</w>') x 1
  ('w', 'i', 'd', 'est</w>') x 1


,word_symbols,count
0,low</w>,1
1,lower</w>,1
2,lowest</w>,1
3,n e w est</w>,1
4,w i d est</w>,1


## 4) Use Learned Merges to Tokenize New Words (greedy, minimal code)
We turn the `merges` list into a rank map (earlier merges have higher priority). Then for each new word:
- Start with `list(chars) + ['</w>']`
- Repeatedly merge the **best-ranked adjacent pair** present
- Stop when no learned pair is applicable; strip `</w>` at the end

In [18]:
if len(merges) == 0:
    print("You haven't learned any merges yet. Re-run the previous BPE step cell a few times.")
else:
    # Build pair->rank dict: lower rank = earlier (higher priority)
    ranks = {m:i for i, m in enumerate(merges)}

    def tokenize_with_merges(word):
        syms = list(word) + ["</w>"]
        # repeatedly merge the best-ranked pair that exists
        while True:
            # list all adjacent pairs
            cand = [((syms[i], syms[i+1]), i) for i in range(len(syms)-1)]
            if not cand:
                break
            # find the best-ranked pair present
            best = None
            best_rank = 10**9
            best_idx = -1
            for (a,b), idx in cand:
                r = ranks.get((a,b), 10**9)
                if r < best_rank:
                    best = (a,b); best_rank = r; best_idx = idx
            if best is None or best_rank == 10**9:
                break
            a,b = best
            syms = syms[:best_idx] + [a+b] + syms[best_idx+2:]
        if syms and syms[-1] == "</w>": syms = syms[:-1]
        return syms

    for w in test_words:
        toks = tokenize_with_merges(w)
        print(f"{w:>8} ->", toks)

    # Compare counts vs. word/char tokenization on one example
    demo = "lowest"
    print("\nComparison on:", demo)
    print("Word-level tokens (split by space):", demo.split())
    print("# tokens (word-level):", len(demo.split()))
    print("Char-level tokens:", list(demo))
    print("# tokens (char-level):", len(list(demo)))
    print("BPE tokens:", tokenize_with_merges(demo))
    print("# tokens (BPE):", len(tokenize_with_merges(demo)))

  lowest -> ['lowest</w>']
     new -> ['n', 'e', 'w']
   widen -> ['w', 'i', 'd', 'e', 'n']
   lower -> ['lower</w>']
    slow -> ['s', 'low</w>']

Comparison on: lowest
Word-level tokens (split by space): ['lowest']
# tokens (word-level): 1
Char-level tokens: ['l', 'o', 'w', 'e', 's', 't']
# tokens (char-level): 6
BPE tokens: ['lowest</w>']
# tokens (BPE): 1
